In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sqlalchemy.orm import sessionmaker
from sqlalchemy import create_engine, not_
from dotenv import load_dotenv
import os
import sys
import requests
from typing import Any

from data.models import Player, GameStats, KtcValue

In [4]:
def fetch(url) -> dict[str, Any]:
    resp = requests.get(url)
    if resp.status_code == 200:
        return resp.json()
    raise ValueError

def get_rosters(league_id) -> dict[str, Any]:
    url = f"https://api.sleeper.app/v1/league/{league_id}/rosters"
    return fetch(url)

def fetch_all_players() -> dict[str, Any]:
    url = "https://api.sleeper.app/v1/players/nfl"
    return fetch(url)

In [5]:
all_players = fetch_all_players()

In [6]:
load_dotenv()
DB_USERNAME = os.environ['DB_USERNAME']
DB_PASSWORD = os.environ['DB_PASSWORD']
DB_HOST = os.environ['DB_HOST']
DB_PORT = os.environ['DB_PORT']
DB_NAME = os.environ['DB_NAME']

DB_URL = f"postgresql://{DB_USERNAME}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"
engine = create_engine(DB_URL, echo=False)

Session = sessionmaker(bind=engine)
session = Session()

In [10]:
missing_players = []
missing_gsis = []

for player_data in all_players.values():
    gsis_id = player_data.get('gsis_id')
    if gsis_id:
        player = session.query(Player).filter_by(gsis_id=gsis_id).first()
        if player:
            player.sleeper_id = str(player_data.get('player_id'))
            player.rotoworld_id = str(player_data.get('rotoworld_id', None))
            player.rotowire_id = str(player_data.get('rotowire_id', None))
            player.yahoo_id = str(player_data.get('yahoo_id', None))
            player.espn_id = str(player_data.get('espn_id', None))
            player.swish_id = str(player_data.get('swish_id', None))
            session.add(player)
        else:
            missing_players.append(player_data['full_name'])
            missing_gsis.append(gsis_id)
session.commit()


In [11]:
players_with_sleeper_id = session.query(Player).filter(Player.sleeper_id.isnot(None)).all()
players_with_sleeper_id

 ...]